In [1]:
from trame_mod import VTKVisualizer
try:
    visualizer = VTKVisualizer(filename="test-files/wave2d.vtk")
except FileNotFoundError as e:
    print(e)

await visualizer.ui.ready
visualizer.ui

HTML(value='<iframe id="trame_trame__template_main" src="http://localhost:63368/index.html?ui=main&reconnect=a…